## Task 3: Customer Segmentation / Clustering

Perform customer segmentation using clustering techniques. Use both profile information (from Customers.csv) and transaction information (from Transactions.csv).
- You have the flexibility to choose any clustering algorithm and any number of clusters in between(2 and 10)
- Calculate clustering metrics, including the DB Index(Evaluation will be done on this).
- Visualise your clusters using relevant plots.

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_customers = pd.read_csv("/home/dispers/PRACHI/Customers.csv")
df_products = pd.read_csv("/home/dispers/PRACHI/Products.csv")
df_txns = pd.read_csv("/home/dispers/PRACHI/Transactions.csv")

df_merged = pd.merge(df_txns, df_products, how = "left", on = ["ProductID","Price"])
df_merged = pd.merge(df_merged, df_customers, how = "left", on = "CustomerID")
df_merged = pd.get_dummies(df_merged, columns=['Region'])

In [ ]:
df_merged['SignupMonth'] = pd.to_datetime(df_merged['SignupDate']).dt.month
df_merged['TransactionMonth'] = pd.to_datetime(df_merged['TransactionDate']).dt.month

In [ ]:
cust_profiles = df_merged.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'TransactionID':'count',
    'Quantity':'sum',
    # 'SignupMonth': 'first',                       # Month of signup
    'TransactionMonth': lambda x: x.mode()[0],    # Top month of transaction
    'Region_Asia': 'first',
    'Region_Europe': 'first',
    'Region_North America': 'first',
    'Region_South America': 'first'
}).reset_index()

# cust_profiles['Region'] = df_merged.groupby('CustomerID')[['Region_Asia', 'Region_Europe', 'Region_North America', 'Region_South America']]
cust_profiles['AvgTransactionValue'] = cust_profiles['TotalValue'] / cust_profiles['TransactionID']
cust_profiles

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns

# # Load data
# customers = pd.read_csv('Customers.csv')
# transactions = pd.read_csv('Transactions.csv')

# # Merge datasets
# df_merged = pd.merge(transactions, customers, on='CustomerID')

# # Aggregate features
# cust_profiles = df_merged.groupby('CustomerID').agg({
#     'TotalValue': 'sum',  # Total transaction value
#     'TransactionID': 'count',  # Number of transactions
#     'Quantity': 'sum',  # Total quantity purchased
#     # 'Age': 'mean',  # Customer's age
#     # 'Income': 'mean'  # Customer's income
# }).reset_index()

# Data Preprocessing
scaler = StandardScaler()
scaled_features = scaler.fit_transform(cust_profiles.iloc[:, 1:])

# Clustering with KMeans
db_scores = []
silhouette_scores = []
inertia_values = []
num_clusters = range(2, 11)

for k in num_clusters:
    kmeans = KMeans(n_clusters=k, random_state=1)
    labels = kmeans.fit_predict(scaled_features)
    db_index = davies_bouldin_score(scaled_features, labels)
    silhouette_avg = silhouette_score(scaled_features, labels)
    inertia = kmeans.inertia_
    
    db_scores.append(db_index)
    silhouette_scores.append(silhouette_avg)
    inertia_values.append(inertia)

# Determine optimal number of clusters (minimizing DB Index)
optimal_k = num_clusters[np.argmin(db_scores)]
print(f"Optimal number of clusters: {optimal_k}")

# Summary of clustering metrics
print("\nClustering Metrics Summary:")
for k, db, sil, inertia in zip(num_clusters, db_scores, silhouette_scores, inertia_values):
    print(f"Clusters: {k}, DB Index: {db:.2f}, Silhouette Score: {sil:.2f}, Inertia: {inertia:.2f}")

# Fit KMeans with optimal clusters
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42)
cust_profiles['Cluster'] = kmeans_optimal.fit_predict(scaled_features)

# Evaluate DB Index for optimal clustering
optimal_db_index = davies_bouldin_score(scaled_features, cust_profiles['Cluster'])
optimal_silhouette = silhouette_score(scaled_features, cust_profiles['Cluster'])

print(f"\nMetrics for {optimal_k} Clusters:")
print(f"Davies-Bouldin Index: {optimal_db_index:.2f}")
print(f"Silhouette Score: {optimal_silhouette:.2f}")

# Visualization of Clusters
plt.figure(figsize=(12, 6))
sns.scatterplot(
    x=cust_profiles['Quantity'],
    y=cust_profiles['TotalValue'],
    hue=cust_profiles['Cluster'],
    palette='viridis',
    s=100
)
plt.title('Customer Segmentation')
plt.xlabel('Quantity Purchased')
plt.ylabel('Total Value')
plt.legend(title='Cluster', loc='upper right')
plt.grid(True)
plt.show()

# Pairplot for feature relationships
sns.pairplot(
    cust_profiles,
    vars=['TotalValue', 'TransactionID', 'Quantity'],
    hue='Cluster',
    palette='viridis'
)
plt.suptitle('Cluster Relationships', y=1.02)
plt.show()

# Save results to a CSV
cust_profiles.to_csv('Customer_Clusters.csv', index=False)
print("Customer_Clusters.csv created successfully!")
